In [1]:
import numpy as np
import xarray as xr
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
physical_devices = tf.config.list_physical_devices("GPU")
#tf.config.experimental.set_memory_growth(physical_devices, True)
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
p = "E:\\Crop_Classification_Phenological\\UAS Precision NCAT files"
crop_type = ["corn", "cotton", "soybean"]
crop_growth_stage = ["2020-06-30_HSI_Sorted", "2020-07-23_HSI_Sorted", "2020-09-04_HSI_Sorted"]
training_hs = []
def create_training_data():
    for cg in crop_growth_stage:
        class_1 = crop_growth_stage.index(cg)
        for ct in crop_type:
            path = os.path.join(p,cg,ct)
            class_2 = crop_type.index(ct)
            for img in os.listdir(path+"\\No_Geo"):
                raster_array = xr.open_rasterio(os.path.join(path+"\\No_Geo",img))
                data = raster_array.values
                #print(data.shape)
                training_hs.append([data, class_1,class_2])
                
create_training_data()
    

C:\Users\bhara\AppData\Local\Temp/ipykernel_8460/3596669406.py:12: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  raster_array = xr.open_rasterio(os.path.join(path+"\\No_Geo",img))


In [4]:
X=[]
y_1=[]
y_2=[]
for features, label1, label2 in training_hs:
    x,y,z = features.shape
    if x==270:
        #print(features[:,0:33,0:39].shape)
        X.append(features[:,0:33,0:39])
        y_1.append(label1)
        y_2.append(label2)
X= np.array(X)

In [5]:
X = X.reshape(-1,270,33*39)

In [6]:
X.shape

(779, 270, 1287)

In [14]:
# -*- coding: utf-8 -*-
"""
Created on Sat Nov 07 00:31:47 2021

@author: bharathi
"""
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Input, Concatenate,SpatialDropout1D
from keras import regularizers
import keras.backend as K
from keras.layers import Activation, Lambda

def channel_normalization(x):
    max_values = K.max(K.abs(x), 2, keepdims=True) + 1e-5
    out = x / max_values
    return out


def wave_net_activation(x):
    tanh_out = Activation('tanh')(x)
    sigm_out = Activation('sigmoid')(x)
    return tf. keras.layers.multiply([tanh_out, sigm_out])


def residual_block(x, s, i, activation, nb_filters, kernel_size, padding, dropout_rate=0, name=''):


    original_x = x
    conv = Conv1D(filters=nb_filters, kernel_size=kernel_size,
                  dilation_rate=i, padding=padding,
                  name=name + '_dilated_conv_%d_tanh_s%d' % (i, s))(x)
    if activation == 'norm_relu':
        x = Activation('relu')(conv)
        x = Lambda(channel_normalization)(x)
    elif activation == 'wavenet':
        x = wave_net_activation(conv)
    else:
        x = Activation(activation)(conv)

    x = SpatialDropout1D(dropout_rate, name=name + '_spatial_dropout1d_%d_s%d_%f' % (i, s, dropout_rate))(x)

    # 1x1 conv.
    x = Conv1D(nb_filters, 1, padding='same')(x)
    res_x = tf.keras.layers.add([original_x, x])
    return res_x, x


def process_dilations(dilations):
    def is_power_of_two(num):
        return num != 0 and ((num & (num - 1)) == 0)

    if all([is_power_of_two(i) for i in dilations]):
        return dilations

    else:
        new_dilations = [2 ** i for i in dilations]
        # print(f'Updated dilations from {dilations} to {new_dilations} because of backwards compatibility.')
        return new_dilations


class TCN:

    def __init__(self,
                 nb_filters=64,
                 kernel_size=2,
                 nb_stacks=1,
                 dilations=None,
                 activation='norm_relu',
                 padding='causal',
                 use_skip_connections=True,
                 dropout_rate=0.0,
                 return_sequences=True,
                 name='tcn'):
        self.name = name
        self.return_sequences = return_sequences
        self.dropout_rate = dropout_rate
        self.use_skip_connections = use_skip_connections
        self.activation = activation
        self.dilations = dilations
        self.nb_stacks = nb_stacks
        self.kernel_size = kernel_size
        self.nb_filters = nb_filters
        self.padding = padding

        if padding != 'causal' and padding != 'same':
            raise ValueError("Only 'causal' or 'same' paddings are compatible for this layer.")


    def __call__(self, inputs):
        if self.dilations is None:
            self.dilations = [1, 2, 4, 8, 16, 32]
        x = inputs
        x = Conv1D(self.nb_filters, 1, padding=self.padding, name=self.name + '_initial_conv')(x)
        skip_connections = []
        for s in range(self.nb_stacks):
            for i in self.dilations:
                x, skip_out = residual_block(x, s, i, self.activation, self.nb_filters,
                                             self.kernel_size, self.padding, self.dropout_rate, name=self.name)
                skip_connections.append(skip_out)
        if self.use_skip_connections:
            x = tf.keras.layers.add(skip_connections)
        x = Activation('relu')(x)

        if not self.return_sequences:
            output_slice_index = -1
            x = Lambda(lambda tt: tt[:, output_slice_index, :])(x)
        return x
def model(input_shape1):
    input1 = Input(input_shape1)
    s1 = Conv1D(64, 270, activation ='relu', input_shape = input_shape1[1:])(input1)
    s2 = Conv1D(64, 270, activation = 'relu', padding = 'same')(s1)
    s3 = Conv1D(64, 270, activation = 'relu', padding = 'same')(s2)
    #s4 = Conv1D(64, 270, activation = 'relu', padding = 'same')(s3)
    #res_x = tf.keras.layers.add([s1, s3])
    #x = Activation('relu')(res_x)

    
    ## Temporal Convolution Modeling
    x = TCN(128,dilations = [1, 2, 4, 8, 16, 32], return_sequences=True, activation = 'wavenet',name = 'tnc1')(s3)
    x = TCN(64,dilations = [1, 2, 4, 8, 16, 32], return_sequences=True, activation = 'wavenet',name = 'tnc2')(x)
    
    z = Flatten()(x)
    y1 = Dense(256, activation= 'relu')(z)
    y1 = Dense(3, activation = 'softmax',name="crop_growth_stage",activity_regularizer=regularizers.l2(0.001))(y1)
    
    y2 = Dense(256,activation = 'relu')(z)
    y2 = Dense(3, activation = 'softmax',name="crop_type",activity_regularizer=regularizers.l2(0.001))(y2)
    model = Model(inputs=input1, outputs=[y1,y2], name="temp_cnn")
    #model = Model(inputs=input1, outputs = y1, name='temp_cnn')
    return model
#model = model((270,1287))
#print(model.summary())

In [15]:
loss_list = {'crop_growth_stage':'categorical_crossentropy', 'crop_type':'categorical_crossentropy'}
metric_list = {'crop_growth_stage':'accuracy', 'crop_type':'accuracy'}
loss_weights = {'crop_growth_stage':1.0,'crop_type':1.0}
#loss_list = {'crop_type':'categorical_crossentropy'}
#metric_list = {'crop_type':'accuracy'}

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=6)

In [ ]:
#MT-TCN
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from keras.utils import np_utils
from numba import cuda
import gc
fold_no = 1
#Y = [y_1, y_2]
acc1_per_fold = []
acc2_per_fold =[]
loss1_per_fold =[]
loss2_per_fold =[]
precision1 =[]
precision2 =[]
recall1,recall2=[],[]
fscore1,fscore2=[],[]
k1_coef, k2_coef = [],[]

nSamples = 779
kfold = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
ID_Inp = np.array(range(nSamples))
#ID_Out = np.array(range(nSamples),3)
y_1 = np.array(y_1)
y_2 = np.array(y_2)
for IDs_Train, IDs_Test in kfold.split(ID_Inp, y_1):
    x_train = X[IDs_Train]
    
    y_train_1, y_train_2 = y_1[IDs_Train], y_2[IDs_Train]
    
    y_train_1 = np_utils.to_categorical(y_train_1,3)
    y_train_2 = np_utils.to_categorical(y_train_2,3)
    
    x_test = X[IDs_Test]
    y_test_1, y_test_2 = y_1[IDs_Test], y_2[IDs_Test]
    
    y_test_1 = np_utils.to_categorical(y_test_1,3)
    y_test_2 = np_utils.to_categorical(y_test_2,3)
    
    neural_network = model((270,1287))
    tf.random.set_seed(1234)

    neural_network.compile(optimizer = opt, loss=loss_list, metrics=metric_list,loss_weights = loss_weights)
    
    print('.............')
    print(f'Training for fold{fold_no}...')
    
    model_history = neural_network.fit(x_train,[y_train_1,y_train_2],batch_size=32,epochs = 50,validation_data = (x_test,[y_test_1,y_test_2]),callbacks=[es],verbose=0)
    
    scores = neural_network.evaluate(x_test,[y_test_1,y_test_2])
    
    results = neural_network.predict(x_test)
    #print(results)
    
    prediction_cg = results[0]
    prediction_ct = results[1]

    pr_cg = prediction_cg
    pr_ct = prediction_ct

    labels_cg = np.argmax(pr_cg, axis=1)
    labels_ct = np.argmax(pr_ct, axis=1)

    y_t1 = np.argmax(y_test_1, axis=1)
    y_t2 = np.argmax(y_test_2, axis=1)


    
    precision, recall, fscore, support = score(labels_ct, y_t1)
    kappa_coefficient_ct = cohen_kappa_score(labels_ct, y_t1)
    
    precision1.append(precision)
    recall1.append(recall)
    fscore1.append(fscore)
    k1_coef.append(kappa_coefficient_ct)

    precision, recall, fscore, support = score(labels_cg, y_t2)
    kappa_coefficient_cg = cohen_kappa_score(labels_cg, y_t2)
    
    precision2.append(precision)
    recall2.append(recall)
    fscore2.append(fscore)
    k2_coef.append(kappa_coefficient_cg)
    
    acc1_per_fold.append(scores[3])
    acc2_per_fold.append(scores[4])
    
    loss1_per_fold.append(scores[1])
    loss2_per_fold.append(scores[2])
    
    fold_no += 1
    del scores,results, x_train,y_train_1,y_train_2,x_test,y_test_1,y_test_2
    gc.collect()
    
#==provide average scores==
print('........')
print('Score per fold')
for i in range(0, len(acc1_per_fold)):
    print('.......')
    print(f'>fold {i+1} - loss{loss1_per_fold[i]}- Accuracy:{acc1_per_fold[i]}%')
    print(f'>fold {i+1} - loss{loss2_per_fold[i]}- Accuracy:{acc2_per_fold[i]}%')
    
print('.......')
print('Average scores for all folds:')
print(f'>Crop Growth Stage Accuracy:{np.mean(acc1_per_fold)}(+-{np.std(acc1_per_fold)})')
print(f'>Crop Type Accuracy:{np.mean(acc2_per_fold)}(+-{np.std(acc2_per_fold)})')


print(f'>Loss:{np.mean(loss1_per_fold)}')
print(f'>Loss:{np.mean(loss1_per_fold)}')
      
print('.......')   



.............
Training for fold1...
5/5 [==============================] - 0s 61ms/step - loss: 0.1735 - crop_growth_stage_loss: 0.0138 - crop_type_loss: 0.1578 - crop_growth_stage_accuracy: 0.9936 - crop_type_accuracy: 0.9423
.............
Training for fold2...
Epoch 00028: early stopping
5/5 [==============================] - 0s 61ms/step - loss: 0.2297 - crop_growth_stage_loss: 0.0592 - crop_type_loss: 0.1687 - crop_growth_stage_accuracy: 0.9679 - crop_type_accuracy: 0.9359
.............
Training for fold3...
5/5 [==============================] - 0s 61ms/step - loss: 0.2519 - crop_growth_stage_loss: 7.4958e-04 - crop_type_loss: 0.2493 - crop_growth_stage_accuracy: 1.0000 - crop_type_accuracy: 0.8526
.............
Training for fold4...
Epoch 00046: early stopping
5/5 [==============================] - 0s 62ms/step - loss: 0.4556 - crop_growth_stage_loss: 0.0465 - crop_type_loss: 0.4073 - crop_growth_stage_accuracy: 0.9936 - crop_type_accuracy: 0.8654
.............
Training for fold5